Some of my favorite resources on transfer learning:
- [Fine-tuning with Keras and Deep Learning](https://www.pyimagesearch.com/2019/06/03/fine-tuning-with-keras-and-deep-learning/)
- [Hands-On Transfer Learning with Python](https://www.amazon.in/Hands-Transfer-Learning-Python-TensorFlow-ebook/dp/B07CB455BF)
- [Applying Deep Transfer Learning for Natural Language Processing (NLP)](https://github.com/dipanjanS/deep_transfer_learning_nlp_dhs2019)

In [6]:
# imports
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from wandb.keras import WandbCallback
from utils import data_utils
import tensorflow as tf
import numpy as np
import wandb
import time

In [2]:
# fix random seed for better reproducibility
tf.random.set_seed(666)

In [3]:
# initialize wandb
wandb.init("ml-bootcamp")

W&B Run: https://app.wandb.ai/sayakpaul/ML-Bootcamp-Launchpad/runs/ct694prq

In [4]:
# don't change this
CLASSES = [b'daisy', b'dandelion', b'roses', b'sunflowers', b'tulips']

In [5]:
# define the constants
BATCH_SIZE = 32
EPOCHS = 20

In [7]:
# let's load up the tfrecord filenames
tfr_pattern_train = "train_tfr/*.tfrec"
train_filenames = tf.io.gfile.glob(tfr_pattern_train)
tfr_pattern_test = "test_tfr/*.tfrec"
test_filenames = tf.io.gfile.glob(tfr_pattern_test)

In [8]:
# create the train and test dataset
training_dataset, steps_per_epoch = data_utils.batch_dataset(train_filenames, BATCH_SIZE, True)
validation_dataset, validation_steps = data_utils.batch_dataset(test_filenames, BATCH_SIZE, False)

In [9]:
# create a utility function to define our model
def create_model(img_size=(224,224), num_class=5, train_base=True):
    input_layer = Input(shape=(img_size[0],img_size[1],3))
    base = VGG16(input_tensor=input_layer,
                    include_top=False,
                    weights="imagenet")
    base.trainable = train_base
    x = base.output
    x = GlobalAveragePooling2D()(x)
    
    preds = Dense(num_class, activation="softmax")(x)
    return Model(inputs=input_layer, outputs=preds)

You might want to go for _layerwise pre-training_ as well. 

In [10]:
# instantiate the model, supply the loss scaled optimizer,
# and compile it
model = create_model()
opt = Adam(learning_rate=1e-4)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=opt,
              metrics=["accuracy"])

58892288/58889256 [==============================] - 2s 0us/step


In [11]:
# train the model
start = time.time()
model.fit_generator(training_dataset, 
    steps_per_epoch=steps_per_epoch,
    validation_data=validation_dataset,
    validation_steps=validation_steps,
    epochs=EPOCHS,
    callbacks=[WandbCallback(data_type="image", labels=CLASSES)])
wandb.log({"training_time": time.time() - start})

Epoch 1/20
97/97 [==============================] - 78s 800ms/step - loss: 1.5380 - accuracy: 0.3012 - val_loss: 1.1774 - val_accuracy: 0.4246
Epoch 2/20
97/97 [==============================] - 68s 699ms/step - loss: 0.9845 - accuracy: 0.6044 - val_loss: 0.5448 - val_accuracy: 0.8125
Epoch 3/20
97/97 [==============================] - 68s 703ms/step - loss: 0.6473 - accuracy: 0.7577 - val_loss: 0.3829 - val_accuracy: 0.8658
Epoch 4/20
97/97 [==============================] - 68s 699ms/step - loss: 0.5481 - accuracy: 0.8038 - val_loss: 0.4505 - val_accuracy: 0.8419
Epoch 5/20
97/97 [==============================] - 68s 703ms/step - loss: 0.3798 - accuracy: 0.8599 - val_loss: 0.2895 - val_accuracy: 0.8768
Epoch 6/20
97/97 [==============================] - 68s 697ms/step - loss: 0.3773 - accuracy: 0.8621 - val_loss: 0.4355 - val_accuracy: 0.8529
Epoch 7/20
97/97 [==============================] - 68s 699ms/step - loss: 0.3749 - accuracy: 0.8621 - val_loss: 0.2017 - val_accuracy: 0.9265

In case you are wondering why training loss is higher than the validation loss [this article](https://www.pyimagesearch.com/2019/10/14/why-is-my-validation-loss-lower-than-my-training-loss/) is an excellent read. 